# FINAL NPZD PROTOTYPE

# NOTE:

actually assemble all variables involved with a flux, and register from there!
add that extra level of complexity! so the problem if one has a dim and another doesn't

here list of vars supplied and flux routing could happen!!!

do this!

also,
think again how to use groups in the cleanest way possible!




In [1]:
import os,sys,inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import phydra

In [2]:
from phydra.components.variables import SV, SVArray

In [3]:
from phydra.components.fluxes import (LinearForcingInput,
                                      LinearMortality,
                                      MonodGrowth,
                                      MultiLimGrowth, Monod_ML,
                                      ListInputFlux
                                     )
from phydra.components.forcings import ConstantForcing, GlobalSlabClimatologyForcing

ImportError: cannot import name 'LinearForcingInput' from 'phydra.components.fluxes' (/Users/bpo/Documents/GitHub/phydra/prototypes/../phydra/components/fluxes/__init__.py)

In [4]:
import numpy as np

np.warnings.filterwarnings('error', category=np.VisibleDeprecationWarning)

In [5]:
NPZD = phydra.create({'Nutrient':SV,
                      'Nutrient2':SV,
                      'Phytoplankton':SV,

                      'Input':LinearForcingInput,                       
                      'Growth':MultiLimGrowth,
                      'Monod':Monod_ML,
                      'Mortality': LinearMortality,
                      
                      # HERE NEED TO ADD THIS:
                      #'N0':GlobalSlabClimatologyForcing,
                      'N0':ConstantForcing,
                      
                      'ListInputFlux':ListInputFlux,
                     })

In [6]:
NPZD

<xsimlab.Model (12 processes, 23 inputs)>
Core
    solver_type       [in]
Time
    time              [in] ('input_time',) sequence of time points ...
Nutrient
    var_label         [in] label / basic state variable
    var_init          [in] initial value / basic state variable
Nutrient2
    var_init          [in] initial value / basic state variable
    var_label         [in] label / basic state variable
Phytoplankton
    var_label         [in] label / basic state variable
    var_init          [in] initial value / basic state variable
N0
    forcing_label     [in] label / 
    value             [in] constant value of forcing
Input
    forcing           [in] label reference / forcing affecting flux
    var               [in] label reference / variable affected by flux
    rate              [in] linear rate of change
Mortality
    var               [in] label reference / variable affected by flux
    rate              [in] linear rate of change
Monod
    resource          [in] label re

In [7]:
odeint_setup = phydra.setup('odeint', NPZD,
            time=np.arange(0,10,0.1),
            input_vars={
                'Nutrient':{'var_label':'N', 'var_init':3.},
                'Nutrient2':{'var_label':'N2', 'var_init':3.},
                'Phytoplankton':{'var_label':'P', 'var_init':0.1},
                
                'Input':{'var':'N', 'rate':0.1, 'forcing':'N0'},
                #'Input2':{'var':'N2', 'rate':0.05, 'forcing':'N0'},
                
                'Growth':{'consumer':'P', 'resource':'N', 'mu_max':1.},
                'Monod':{'resource':'N', 'halfsat':1.5},
                
                'Mortality':{'var':'P', 'rate':0.1},
                
                'N0':{'forcing_label':'N0', 'value':2.},
                #'N0':{'forcing_label':'N0', 'dataset':'n0x',
                #      'lat':47, 'lon':-20, 'rbb':2,
                #      'smooth':5, 'k':2},
                
                'ListInputFlux':{'resources':['N', 'N2'], 'consumer':'P', 'halfsats':[2.,2.]},
            },
            output_vars="ALL"
            )

In [ ]:
with NPZD:
     odeint_out = odeint_setup.xsimlab.run()

In [ ]:
odeint_out

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
time = odeint_out.Time__time

plt.plot(time, odeint_out.Nutrient__var_value, label='N')
#plt.plot(time, odeint_out.Nutrient2__var_value, label='N2')
plt.plot(time, odeint_out.Phytoplankton__var_value.T, label='P')
#plt.ylim(bottom=0)
plt.legend()

In [ ]:
plt.plot(time, odeint_out.Input__input_value, label='N Input')

In [ ]:
plt.plot(time, odeint_out.Input__input_value, label='N Input')
plt.plot(time, odeint_out.Growth__growth_value.T, label='P ListINPUT Growth')
plt.plot(time, -odeint_out.Mortality__death_value, label='P Mortality')
plt.legend()

In [ ]:
stepwise_setup = phydra.update_setup(model=NPZD, old_setup=odeint_setup, new_solver="stepwise")

In [ ]:
with NPZD:
    stepwise_out = stepwise_setup.xsimlab.run()

In [ ]:
stepwise_out

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
time = stepwise_out.Time__time

plt.plot(time, stepwise_out.Nutrient__var_value, label='N')
#plt.plot(time, stepwise_out.Nutrient2__var_value, label='N2')
plt.plot(time, stepwise_out.Phytoplankton__var_value.T, label='P')
plt.ylim(bottom=0)
plt.legend()

In [ ]:
plt.plot(time, stepwise_out.Input__input_value, label='N Input')
plt.ylim(bottom=0)
plt.legend()

In [8]:
gekko_setup = phydra.update_setup(model=NPZD, 
                                  old_setup=odeint_setup, 
                                  new_solver="gekko", 
                                  # gekko solver works better (much faster) with smaller time steps:
                                  # new_time=np.arange(0,201,.1)
                                 )

In [9]:
with NPZD:
    gekko_out = gekko_setup.xsimlab.run()

initializing model backend
Initializing Model Time
Initializing component Nutrient
Initializing component Nutrient2
Initializing component Phytoplankton
Initializing component N0
forcing function is in directory: /Users/bpo/Documents/GitHub/phydra/prototypes
forcing_val: 2.0
Initializing component Input
Initializing component Mortality
Initializing component Monod
Initializing component ListInputFlux
{'consumer': 0.1, 'resources': array([3.0, 3.0], dtype=object), 'halfsats': array([2.0, 2.0], dtype=object)}
(),(),(m),(m)->(m)
[3.0 3.0] p [2.0 2.0]
((0+(n+listinputflux_halfsats0))+(n2+listinputflux_halfsats1))
out: [((((n)/(((0+(n+listinputflux_halfsats0))+(n2+listinputflux_halfsats1)))))*(p))
 ((((n2)/(((0+(n+listinputflux_halfsats0))+(n2+listinputflux_halfsats1)))))*(p))] (2,)
Initializing component Growth
assembling model
SOLVER : <phydra.backend.solvers.GEKKOSolver object at 0x1583fb0a0>
Model dicts are assembled:
Model contains: 
Variables:['time', 'N', 'N2', 'P'] 
Parameters:['N0_

In [10]:
time = gekko_out.Time__time
plt.plot(time, gekko_out.Nutrient__var_value, label='N')
#plt.plot(time, gekko_out.Nutrient2__var_value, label='N2')
plt.plot(time, gekko_out.Phytoplankton__var_value.T, label='P')
plt.ylim(bottom=0)
plt.legend()

NameError: name 'plt' is not defined

In [11]:
plt.plot(time, gekko_out.Input__input_value, label='N_input')
plt.ylim(bottom=0)
plt.legend()

NameError: name 'plt' is not defined